# Movement around Corners

In the following we'll investigate the movement of pedestrians around corners. When pedestrians walk around corners they are expected to slow down and take a path that is close to the corner. According to RiMEA Test 6 **[TODO REF]** a scenario is configured where **20 agents** move towards a **corner** at which they should turn to the left.

Let's begin by importing the required packages for our simulation:

In [ ]:
from shapely import GeometryCollection, Polygon, to_wkt
import pathlib
import jupedsim as jps
import pedpy
import pandas as pd
import numpy as np
import plotly  # visualise trajectories
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs import Figure
import sqlite3
from numpy.random import normal  # normal distribution of free movement speed

## Setting up the Geometry

We define a corridor with a width of 2 meters and a corner on halfway:

In [ ]:
area = Polygon([(0, 0), (12, 0), (12, 12), (10, 12), (10, 2), (0, 2)])
area

## Definition of Start Positions and Exit

Now we'll calculate the position of 20 agents in the lower left part of the geometry within an rectangle of 6 x 2 meters. To calculate the positions we use a library function from JuPedSim. We assume an agent size of 0.3 m and set the distance parameters accordingly. The exit is defined in the upper right of the geometry.

In [ ]:
spawning_area = Polygon([(0, 0), (6, 0), (6, 2), (0, 2)])
num_agents = 20
positions = jps.distributions.distribute_by_number(
    polygon=spawning_area,
    number_of_agents=num_agents,
    distance_to_agents=0.4,
    distance_to_polygon=0.15,
    seed=1,
)
exit_area = Polygon([(10, 11), (12, 11), (12, 12), (10, 12)])

**TODO: plot config setup - geo + start positions + exit**

## Setting up the Simulation and Routing Details

As a next step we create a simulation object, set the configuration for the operational model (collision-free speed model) and define the routes for the agents. For this scenario only one journey is created as all agents should follow the same route.

In [ ]:
trajectory_file = "corner.sqlite"  # output file
simulation = jps.Simulation(
    model=jps.CollisionFreeSpeedModel(),
    geometry=area,
    trajectory_writer=jps.SqliteTrajectoryWriter(
        output_file=pathlib.Path(trajectory_file)
    ),
)

In [ ]:
exit_id = simulation.add_exit_stage(exit_area.exterior.coords[:-1])
journey = jps.JourneyDescription([exit_id])
journey_id = simulation.add_journey(journey)

## Specifying Agent Parameters

As a next step we define the model-specific parameters for the agents. They share the same journey and model parameters except for the free movement speed which is normally distributed. 

In [ ]:
v_distribution = normal(1.34, 0.2, num_agents)

## Executing the Simulation

Now we can specifiy the indiviual starting positions and speeds and add the agents to the simulation. After that the simulation is started and iterates until all agents have reached the exit.

In [ ]:
for position, v0 in zip(positions, v_distribution):
    simulation.add_agent(
        jps.CollisionFreeSpeedModelAgentParameters(
            journey_id=journey_id, stage_id=exit_id, position=position, v0=v0
        )
    )

while simulation.agent_count() > 0:
    simulation.iterate()

## Visualizing the Trajectories


In [ ]:
from jupedsim.internal.notebook_utils import animate, read_sqlite_file

trajectory_data, walkable_area = read_sqlite_file(trajectory_file)
animate(trajectory_data, walkable_area)

**TODO add colorbar for speed to plot**

As expected the agents choose the shortest path and approach the corner in a funnel-shaped formation. Agents moving closer to the corner become slower than agents at the edge of the crowd who choose a longer path around the corner.

## References & Further Exploration

**TODO RiMEA reference**

The chosen model here is based on the collision-free speed model. JuPedSim also incorporates another model known as GCFM. For more details on GCFM, refer to another notebook **(TODO: Link to the GCFM notebook)**.

The demonstration employed a straightforward journey with a singular exit. For a more intricate journey featuring multiple intermediate stops and waiting zones, see the upcoming section **(TODO: Link to the advanced journey section)**.